# Computational Methods in Economics

## Problem Set - Root Finding: Suggested Solutions

In [1]:
# Author: Alex Schmitt (schmitt@ifo.de), Christina Littlejohn (littlejohn@ifo.de)

import datetime
print('Last update: ' + str(datetime.datetime.today()))

Last update: 2018-12-12 14:19:29.022465


### Preliminaries

#### Import Modules

In [2]:
import numpy as np
import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

## Question 1 (N)

Write a function **mybisect(f, a, b)** in Python that implements the pseudo-code for the bisection method from the lecture. Then, test it on the function 
\begin{equation*}
    f(x) = \sin(4 (x - 1/4)) + x + x^{20} - 1,
\end{equation*}
i.e. find a root of this function. Compare your result to what SciPy's in-built function returns. 

*Hint*: most modern programming languages have some type of **while**-loop, which will prove useful here. Moreover, in Python/NumPy, consider using the **abs()** and **np.sign()** functions.  

In [3]:
# function to use bisection on
def fun(x):
    return np.sin(4 * (x - 0.25)) + x + x**20 - 1

In [4]:
def mybisect(fun, a, b, maxit = 1000):
    """
    Implements the bisection method
    """
    ## check on signs of function values at initial points a and b
    assert np.sign(fun(a)) == - np.sign(fun(b)), "Choose a and b such that f(a) and f(b) have different signs"
    
    # iteration counter
    it = 0
    # choose tolerance level
    tol = 1e-10
    # initialize d (= function value at x)
    d = 1
    # while-loop: iterate until d sufficiently small
    while abs(d) > tol and it < maxit:
        it += 1
        # find intermediate value between a and b
        x = (a + b)/2
        # evaluate function
        d = fun(x)
        # find new end points for interval [a,b]
        if np.sign(d) == np.sign(fun(a)):
            a = x
        elif np.sign(d) == np.sign(fun(b)):
            b = x
    
    print("Number of iterations = {}".format(it) )
    
    return x

print(mybisect(fun,0,2))        
print(scipy.optimize.bisect(fun,0,2))

Number of iterations = 29
0.408293504267931
0.4082935042806639


In [5]:
## check for different starting values -> will throw an error
# print(mybisect(fun,-3,2))  

## Question 2 (N)

Solve the example about the Cournot Duopoly in M&F, p. 35 ff., in Python using Newton's method, and compare your result to M&F.

In [6]:
def cournot(x):
    """
    Implements a system of equations in two unknowns, here the Cournot Duopoly model
    f_i(q1, q2) = [(q1 + q2)^(-1/eta) - (1/eta)*[(q1 + q2)^(-1/eta - 1)]*q_i - c_i*q_i]
    """
    c = [0.6, 0.8]
    eta = 1.6
    e = -1/eta
    
    return np.array(( (x[0]+x[1])**e + e*((x[0]+x[1])**(e-1))*x[0] - c[0]*x[0],
                    (x[0]+x[1])**e + e*((x[0]+x[1])**(e-1))*x[1] - c[1]*x[1]))

In [7]:
def fun_J(x):
    """
    Implements the Jacobian system of equation in two unknowns above
    """
    c = [0.6, 0.8]
    eta = 1.6
    e = -1/eta
    
    f_00 = (e*(x[0]+x[1])**(e-2))*(e*x[0] + x[0] + 2*x[1]) - c[0] 
    f_01 = (e*(x[0]+x[1])**(e-2))*(e*x[0] + x[1])
    f_10 = (e*(x[0]+x[1])**(e-2))*(e*x[1] + x[0])
    f_11 = (e*(x[0]+x[1])**(e-2))*(e*x[1] + 2*x[0] + x[1]) - c[1]
    
    return np.array([[f_00, f_01], [f_10, f_11]])

In [8]:
def my_newton_mult(fun, fun_J, x,  tol = 1e-8, tol2 = 1e-6, maxit = 1000):
    """
    Implements Newton's method for a vector-valued function
    """    
    dist = 1
    it = 0
    while dist > tol and it < maxit:
        it += 1
        f, J = fun(x), fun_J(x)
        x_new = x - np.linalg.inv(J) @ f
        dist = np.linalg.norm(x - x_new) / (1 + np.linalg.norm(x))
        x = x_new
    
    print("Number of iterations = {}".format(it) )
    
    if np.linalg.norm(fun(x)) < tol2: 
        return x
    else:
        print("No solution found!")
    
    return x

In [9]:
x_init = [0.2, 0.2]
x = my_newton_mult(cournot, fun_J, x_init)
print(x)

Number of iterations = 6
[0.8395676  0.68879643]


## Question 3 (N)

Modify the pseudo-code for Newton's method to include backstepping, as outlined in the lecture. Then, include backstepping to the Python function **my_newton**. 

Pseudo-code for Newton's method with backstepping:

(i) Specify tolerance levels $tol1$ and $tol2$ and choose a starting guess $x^{(0)}$.

(ii) Compute the suggested step as 

\begin{equation}
 s^{(k)} = - J(x^{(k)})^{-1} f(x^{(k)})
\end{equation}

(iii) If $\left| \left|\ f(x^{(k)} + s^{(k)}) \right| \right|$ < $\left| \left|\ f(x^{(k)}) \right| \right|$, set $x^{(k+1)} = x^{(k)} + s^{(k)}$ and go to (vi).

(iv) Otherwise, if $\left| \left|\ f\left(x^{(k)} + \frac{s^{(k)}}{2}\right) \right| \right|$ < $\left| \left|\ f(x^{(k)} + s^{(k)}) \right| \right|$, set $s^{(k)} = \frac{s^{(k)}}{2}$ and go back to (iii).

(v) Otherwise, set $x^{(k+1)} = x^{(k)} + s^{(k)}$.

(vi) Check the stopping rule: if $\left| \left|\ x^{(k+1)} - x^{(k)}) \right| \right| < tol1$, stop. If not, go back to (ii).

(vii) If $\left| \left|\ f(x^{(k)}) \right| \right| < tol2$, report $x^{(k+1)}$ as the solution. Otherwise, report failure. 

In [10]:
def my_newton_bs(fun, fun_d, x,  tol1 = 1e-8,  tol2 = 1e-8, maxit = 100):
    """
    Implements Newton's method for a vector-valued function with backstepping
    """    
    dist = 1
    it = 0
    
    while dist > tol1 and it < maxit:
        it += 1
        f, J = fun(x), fun_d(x)
        ## step (ii): compute suggested step
        s = - np.linalg.inv(J) @ f
        
        ## start backstepping routine
        while np.linalg.norm( fun(x + s) ) > np.linalg.norm( fun(x) ): ## step (iii)
            if np.linalg.norm( fun(x + 0.5 * s) ) < np.linalg.norm( fun(x + s) ): # step (iv)
                s = 0.5 * s
            else:   # step (v)
                break ## terminate inner while loop 
        
        x_new = x + s
        dist = np.linalg.norm(x - x_new) / (1 + np.linalg.norm(x))
        x = x_new
    
    print("Number of iterations = {}".format(it) )
    
    if np.linalg.norm( fun(x) ) < tol2:
        return x
    else:
        print('No solution found!')


Compare the example from the lecture:

In [11]:
def fun_vv(x):
    """
    Implements a system of equation in two unknowns, here f(x) = [x2**2 - 1; sin(x1) - x2]
    """
    return np.array( (x[1]**2 - 1 , np.sin(x[0]) - x[1] ) )


def fun_J(x):
    """
    Implements the Jacobian system of equation in two unknowns above
    """
    f_00 = 0
    f_01 = 2 * x[1]
    f_10 = np.cos(x[0])
    f_11 = -1
    
    return np.array([[f_00, f_01], [f_10, f_11]])  

        
x_init = [1.5,0.9]
x = my_newton_bs(fun_vv, fun_J, x_init)
print(x)

Number of iterations = 22
[1.57079635 1.        ]


## Question 4 (N)

Write a function **mysecant(f, x0)** in Python that implements the pseudo-code for the secant method from the lecture. Again, test it on the function $f$ and compare the result to question 1.

In [12]:
def g_secant(fun, f_old, x, x_old):
    """
    Implements the update rule for the secant method
    """
    f = fun(x)
    fd = (f - f_old) / (x - x_old)

    return x - f * fd**(-1), f

In [13]:
def my_secant(fun, x, tol1 = 1e-8,  tol2 = 1e-8, maxit = 100):
    """
    Implements the secant method for a univariate scalar function
    """        
    dist = 1
    it = 0
    
    x_old = 0.9 * x
    f_old = fun(x_old)

    while dist > tol1 and it < maxit:
        it += 1
        x_new, f = g_secant(fun, f_old, x, x_old)
        dist = abs(x - x_new) / (1 + abs(x))
        
        ## store "old" function and x value for next iteration
        f_old = f
        x_old = x
        
        x = x_new
        print(x_new)

    print("Number of iterations = {}".format(it) )
    
    if abs(fun(x)) < tol2: 
        return x
    else:
        print("No solution found!")

In [14]:
%timeit -r1 -n1 my_secant(fun, 0.01)

0.5603210327501603
0.4399462903116683
0.39893802965620273
0.40864779800453077
0.40829712124132966
0.40829350284198784
0.4082935042793729
Number of iterations = 7
693 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Running Scipy's **scipy.optimize.newton** (without providing a derivative) also applies the secant method:

In [15]:
scipy.optimize.newton(fun, 0.01)

0.408293504279372

## Question 5 (N)

Solve question 3.7 in M&F, p. 55.

The problem of agent $i$ is given by

\begin{equation}
    \max U_i(x) = \sum_{j = 1}^2 a_{ij} (v_{ij} + 1)^{-1} x_{ij}^{v_{ij} + 1} 
\end{equation}

s.t. $p_1 x_{i1} + p_2 x_{i2} = p_1 e_{i1} + p_2 e_{i2}$. 


Taking first-conditions gives 

\begin{equation}
    \frac{\partial U_i(x)}{\partial x_{ij}} = U_{ij}(x) = a_{ij} x_{ij}^{v_{ij}} = \lambda_i p_j
\end{equation},

where $\lambda_i$ is the Lagrange multiplier corresponding to agent $i$.

Hence, we have three first order conditions, for $i \in \{1, 2, 3\}$:

\begin{equation}
     \frac{ a_{i1} x_{i1}^{v_{i1}} }{p_1} - \frac{ a_{i2} x_{i2}^{v_{i2}} }{p_2} = 0
\end{equation},

Moreover, we have three budget constraints:

\begin{equation}
    p_1 x_{i1} + p_2 x_{i2} - p_1 e_{i1} - p_2 e_{i2} = 0
\end{equation}

Finally, the markets clear, implying two resource constraints:

\begin{equation}
    \sum_{i} x_{i1} = \sum_{i}  e_{i1},\quad \sum_{i} x_{i2} = \sum_{i}  e_{i2}
\end{equation}

In total, we have a system of 8 equations - three f.o.c.'s, three budget constraints and two resource constraints - in 7 variables, six quantities $x_{ij}$ and the price $p_1$ (recall that $p_2 = 1 - p_1$). Of course Walras' law applies: in the context of this question, given that the agents' budget constraints all hold with equality, if we have an equilibrium on one market (i.e. no excess demand or supply), we also have an equilibrium on the other market (make sure you are able to verify this!).

Hence, we have to use only 7 of the 8 equations, without changing the problem. Note that applying Walras' law here is actually necessary from a purely computational point of view: recall that (Quasi-)Newton methods are based on solving a *square* system of linear equations.


In [16]:
## read parameters
A = np.array([ [2.0, 1.5],
               [1.5, 2.0],
               [1.5, 2.0]  ])

V = np.array([ [-2.0, -0.5],
               [-1.5, -0.5],
               [-0.5, -1.5] ])

E = np.array([ [2.0, 3.0],
               [1.0, 2.0],
               [4.0, 0.0]  ])

In [17]:
## define residual function

def S(z, A, V, E):
    
    X = np.array([ [np.exp(z[0]), np.exp(z[1])],
               [np.exp(z[2]), np.exp(z[3])],
               [np.exp(z[4]), np.exp( z[5])]  ])
    p1 = np.exp( z[6])
    
    S = np.zeros(7)
    
    ### f.o.c.'s
    for i in range(3):
        S[i] = A[i, 0] * X[i, 0]**V[i, 0] / p1 - A[i, 1] * X[i, 1]**V[i, 1] / (1 - p1)
    
    ## budget constraints
    for i in range(3):
        S[3 + i] = p1 * X[i, 0] + (1 - p1) * X[i, 1] - (p1 * E[i, 0] + (1 - p1) * E[i, 1])
    
    
    ## resource constraint good 1 and 2
    S[6] = np.sum(X[:, 0], axis = 0) - np.sum(E[:, 0], axis = 0)
    
    return S

In [18]:
## initial guess - recall that variables are in logs!
z0 = np.array([1, 1, 0.5, 0.5, 0.5, 0.5, -1])
## solve the problem
res = scipy.optimize.root(S, z0, method = 'broyden1', args = (A, V, E))
print(res)
print( np.exp( res.x ) )

     fun: array([-7.16270507e-08,  1.24932218e-07,  2.75794282e-06, -1.06633794e-07,
       -4.26687452e-07,  8.80209219e-08, -1.98529451e-06])
 message: 'A solution was found at the specified tolerance.'
     nit: 32
  status: 1
 success: True
       x: array([ 1.06485824,  1.01624014,  0.86227772,  0.4943689 ,  0.54871128,
       -0.51458403, -1.56775563])
[2.90042778 2.76278752 2.36854945 1.63946325 1.73102078 0.59774919
 0.20851264]


Finally, note the approach above is just one of several possible. In particular, we reduce the dimension of the solution vector by substituting the budget and resource constraints (rather than using them as functions in our system). In the extreme case, you can reduce the system to three equations in three unknown variables ($x_{11}, x_{21}, p1$).

However, while this is fine conceptually, it is not recommended from a numerical perspective. In particular, we run the risk sure that the variables computed inside the function (rather than solved for directly) are negative. Hence, reducing the dimension of the solution vector does not reduce coding time, and can be quite unstable numerically.   

In [19]:
## define alternative residual function (with three arguments)

def S_alt(z, A, V, E):
    
    x11 = np.exp(z[0])
    x21 = np.exp(z[1])
    p1 = np.exp(z[2])

    S = np.zeros(3)
    
    ## derived variables
    ## use budget constraints for to get remaining quantities for consumers 1 and 2
    x12 = (-p1 * x11 + (p1 * E[0, 0] + (1 - p1) * E[0, 1]) ) / (1 - p1) 
    x22 = (-p1 * x21 + (p1 * E[1, 0] + (1 - p1) * E[1, 1]) ) / (1 - p1) 

    ## use resource constraints to get quantities for consumer 3
    x31 = np.sum(E[:, 0], axis = 0) - x11 - x21
    x32 = np.sum(E[:, 1], axis = 0) - x12 - x22

    ### f.o.c.'s

    S[0] = A[0, 0] * x11**V[0, 0] / p1 - A[0, 1] * x12**V[0, 1] / (1 - p1)
    S[1] = A[1, 0] * x21**V[1, 0] / p1 - A[1, 1] * x22**V[1, 1] / (1 - p1)
    S[2] = A[2, 0] * x31**V[2, 0] / p1 - A[2, 1] * x32**V[2, 1] / (1 - p1)
    
    return S

In [20]:
## initial guess - use corresponding values found above and recall that variables are in logs!
z0_alt = np.array([np.log(2.90042778), np.log(2.36854945), np.log(0.20851264)])
S_alt(z0_alt, A, V, E)

array([-6.73597544e-08,  4.05430368e-07, -9.05020796e-06])

In [23]:
## solve the problem -> NOTE THAT WE DON'T GET CONVERGENCE HERE, EVEN WHEN STARTING AT THE RIGHT VALUES!
# res = scipy.optimize.root(S_alt, z0_alt, method = 'broyden2', args = (A, V, E))
# print(res)
# print( np.exp( res.x ) )

## Question 6 (A)

(a) Show that the update rule for $A^{(k+1)}$ used in Broyden's method,

\begin{equation}
 A^{(k+1)} = A^{(k)} + \frac{ \left( \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)}) - A^{(k)} \mathbf{p}^{(k)} \right) (\mathbf{p}^{(k)})^T}{(\mathbf{p}^{(k)})^T \mathbf{p}^{(k)}}
\end{equation}

satisfies the secant condition,

\begin{equation}
 A^{(k+1)} \mathbf{p}^{(k)} = \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)}).
\end{equation}

 and

\begin{equation}
 A^{(k+1)} \mathbf{q} = A^{(k)} \mathbf{q}\ \ \text{for}\ \ \mathbf{q}^{T} \mathbf{p}^{(k)} = 0
\end{equation}.

(b) To prepare question (c), Show that for any vector $\mathbf{p} \in \mathbb{R}^n$, we have 

\begin{equation}
    \left| \left| \frac{\mathbf{p}\ \mathbf{p}^T}{\mathbf{p}^T \mathbf{p} } \right| \right| = 1
\end{equation}


(c) Using the result from question (b), show that

\begin{equation}
 A^{(k+1)} \in \arg \min_{A :\ A \mathbf{p}^{(k)} = \mathbf{f}(\mathbf{x}^{(k+1)}) - \mathbf{f}(\mathbf{x}^{(k)})} ||\ A - A^{(k)} ||
\end{equation}

Hint: Use the update rule in (a) to rewrite the distance $||\ A^{(k+1)}  - A^{(k)} ||$.